Whisper models:
* [OpenAI Whisper - installation requirements](https://github.com/openai/whisper)
* [Hugging Face Transformers - Whisper](https://huggingface.co/openai/whisper-large-v3/blob/main/README.md)

In [ ]:
!pwd

In [ ]:
#aa added
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate

In [ ]:
# Imports
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline
from IPython.display import Audio

In [ ]:
# aa: changed audio file path
# Config params
pipeline_id: str = "automatic-speech-recognition"
model_id: str = "openai/whisper-large-v3"
device: str = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
audio_file: str = r"./grip_audio_file.mp3"

#### Instantiate model and spreech processing pipeline
***

In [ ]:
# Instantiate the model
model = WhisperForConditionalGeneration.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)


In [ ]:
# Instantiate processors (which includes the tokenizer and feature extractor for Whisper)
processor = WhisperProcessor.from_pretrained(model_id)

# Instantiate the pipeline
pipe = pipeline(
    pipeline_id,
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

#### Run inference
***

In [ ]:
# play audio
Audio(audio_file)

In [ ]:
!curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz

In [ ]:
ffmdir = !find . -iname ffmpeg-*-static
path = %env PATH
path = path + ':' + ffmdir[0]
%env PATH $path
!which ffmpeg
print('Done!')

In [ ]:
%%time 
# aa: added previous %%time
# Run transcription
generate_kwargs = {
    "language": "german",
    "task": "transcribe",
    "condition_on_prev_tokens": True,
    "temperature": (0.0, 0.2, 0.4, 0.6, 0.8, 1.0),
    "logprob_threshold": -1.0,
    "no_speech_threshold": 0.6,
}

# run transcription 
try:
    #aa added return_timestamps=True
    result = pipe(audio_file, generate_kwargs=generate_kwargs, return_timestamps=True)

    # play audio
    display(Audio(audio_file))

    # show transcribed text
    text = result["text"]
    print(f"Transcribed text:\n\t{text}")
    
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
%%time   
#aa: added previous %%time
# Run transcription
generate_kwargs = {
    "language": "german",
    "task": "translate",
    "condition_on_prev_tokens": True,
    "temperature": (0.0, 0.2, 0.4, 0.6, 0.8, 1.0),
    "logprob_threshold": -1.0,
    "no_speech_threshold": 0.6,
}

# run transcription 
try:
    #aa added return_timestamps=True
    result = pipe(audio_file, generate_kwargs=generate_kwargs, return_timestamps=True)

    # play audio
    display(Audio(audio_file))

    # show transcribed text
    text = result["text"]
    print(f"Translated text:\n\t{text}")
    
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
#aa: convert the model to onnx

In [1]:
!pip install optimum[exporters]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.7 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 113.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 134.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.0
    Uninstalling transformers-4.48.0:
      Successfully uninstalled transformers-4.48.0

[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!optimum-cli export onnx --help

/opt/app-root/lib64/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/app-root/lib64/python3.9/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue:

In [3]:
!mkdir -p models2

In [ ]:
%%time
!optimum-cli export onnx --model openai/whisper-large-v3 models/

In [6]:
!optimum-cli export onnx --model openai/whisper-small  models2

/opt/app-root/lib64/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/app-root/lib64/python3.9/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue:

In [7]:
#not needed pytorch image has already these packages
#!pip install boto3 botocore

In [10]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
   raise ValueError("One or more connection variables are empty.  "
                    "Please check your connection to an S3 bucket.")
        
session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)
        
s3_resource = session.resource(
   's3',
   config=botocore.client.Config(signature_version='s3v4'),
   endpoint_url=endpoint_url,
   region_name=region_name)
        
bucket = s3_resource.Bucket(bucket_name)
        
        
def upload_directory_to_s3(local_directory, s3_prefix):
  num_files = 0
  for root, dirs, files in os.walk(local_directory):
     for filename in files:
        file_path = os.path.join(root, filename)
        relative_path = os.path.relpath(file_path, local_directory)
        s3_key = os.path.join(s3_prefix, relative_path)
        print(f"{file_path} -> {s3_key}")
        bucket.upload_file(file_path, s3_key)
        num_files += 1
  return num_files
                                                        
                                                        
def list_objects(prefix):
   filter = bucket.objects.filter(Prefix=prefix)
   for obj in filter.all():
      print(obj.key)

In [11]:
list_objects("models")

In [12]:
local_models_directory = "models"

if not os.path.isdir(local_models_directory):
   raise ValueError(f"The directory '{local_models_directory}' does not exist.  "
                     "Did you finish training the model in the previous notebook?")
        
num_files = upload_directory_to_s3("models", "models")
        
if num_files == 0:
   raise ValueError("No files uploaded.  Did you finish training and "
                    "saving the model to the \"models\" directory?  "
                    "Check for \"models/fraud/1/model.onnx\"")

models/config.json -> models/config.json
models/generation_config.json -> models/generation_config.json
models/tokenizer_config.json -> models/tokenizer_config.json
models/special_tokens_map.json -> models/special_tokens_map.json
models/added_tokens.json -> models/added_tokens.json
models/vocab.json -> models/vocab.json
models/merges.txt -> models/merges.txt
models/normalizer.json -> models/normalizer.json
models/tokenizer.json -> models/tokenizer.json
models/preprocessor_config.json -> models/preprocessor_config.json
models/encoder_model.onnx -> models/encoder_model.onnx
models/decoder_model.onnx -> models/decoder_model.onnx
models/decoder_with_past_model.onnx -> models/decoder_with_past_model.onnx
models/decoder_model_merged.onnx -> models/decoder_model_merged.onnx


In [13]:
list_objects("models")

models/added_tokens.json
models/config.json
models/decoder_model.onnx
models/decoder_model_merged.onnx
models/decoder_with_past_model.onnx
models/encoder_model.onnx
models/generation_config.json
models/merges.txt
models/normalizer.json
models/preprocessor_config.json
models/special_tokens_map.json
models/tokenizer.json
models/tokenizer_config.json
models/vocab.json


In [ ]:
!curl -s https://cc-cc.apps.ocp4.example.com/v2/models

In [ ]:
!pwd

In [ ]:
from transformers import WhisperForConditionalGeneration
hf_model_id = "openai/whisper-medium" # Change to your model url
hf_model = WhisperForConditionalGeneration.from_pretrained(hf_model_id)
torch.save(merged_model, "./pretrained_model_path")

In [1]:
!pip install optimum transformers onnxruntime


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
model_id = "openai/whisper-tiny"
from optimum.onnxruntime import ORTModelForSpeechSeq2Seq
model = ORTModelForSpeechSeq2Seq.from_pretrained('onnx_variant', export=True)
model.save_pretrained('./onnx_model')

In [ ]:
from transformers import WhisperForConditionalGeneration

model_name = "openai/whisper-medium"
model = WhisperForConditionalGeneration.from_pretrained(model_name)

In [ ]:
model.save_pretrained('./onnx_model')

In [3]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
   raise ValueError("One or more connection variables are empty.  "
                    "Please check your connection to an S3 bucket.")
        
session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)
        
s3_resource = session.resource(
   's3',
   config=botocore.client.Config(signature_version='s3v4'),
   endpoint_url=endpoint_url,
   region_name=region_name)
        
bucket = s3_resource.Bucket(bucket_name)
        
        
def upload_directory_to_s3(local_directory, s3_prefix):
  num_files = 0
  for root, dirs, files in os.walk(local_directory):
     for filename in files:
        file_path = os.path.join(root, filename)
        relative_path = os.path.relpath(file_path, local_directory)
        s3_key = os.path.join(s3_prefix, relative_path)
        print(f"{file_path} -> {s3_key}")
        bucket.upload_file(file_path, s3_key)
        num_files += 1
  return num_files
                                                        
                                                        
def list_objects(prefix):
   filter = bucket.objects.filter(Prefix=prefix)
   for obj in filter.all():
      print(obj.key)

In [4]:
list_objects("models")

In [5]:
local_models_directory = "models"

if not os.path.isdir(local_models_directory):
   raise ValueError(f"The directory '{local_models_directory}' does not exist.  "
                     "Did you finish training the model in the previous notebook?")
        
num_files = upload_directory_to_s3("models", "models")
        
if num_files == 0:
   raise ValueError("No files uploaded.  Did you finish training and "
                    "saving the model to the \"models\" directory?  "
                    "Check for \"models/fraud/1/model.onnx\"")

models/config.json -> models/config.json
models/generation_config.json -> models/generation_config.json
models/tokenizer_config.json -> models/tokenizer_config.json
models/special_tokens_map.json -> models/special_tokens_map.json
models/added_tokens.json -> models/added_tokens.json
models/vocab.json -> models/vocab.json
models/merges.txt -> models/merges.txt
models/normalizer.json -> models/normalizer.json
models/tokenizer.json -> models/tokenizer.json
models/preprocessor_config.json -> models/preprocessor_config.json
models/encoder_model.onnx -> models/encoder_model.onnx
models/decoder_model.onnx -> models/decoder_model.onnx


In [1]:
from transformers import WhisperConfig
from optimum.exporters.onnx import main_export
from optimum.exporters.onnx.model_configs import WhisperOnnxConfig

model_id = "openai/whisper-small"

print("Exporting model as ONNX")

config = WhisperConfig.from_pretrained(model_id)
onnx_config = WhisperOnnxConfig(config, task="automatic-speech-recognition")

encoder_config = onnx_config.with_behavior("encoder")
decoder_config = onnx_config.with_behavior("decoder")

custom_onnx_configs={
        "encoder_model": encoder_config,
        "decoder_model": decoder_config
}

main_export(
        model_id,
        output="models",
        task="automatic-speech-recognition",
        custom_onnx_configs=custom_onnx_configs
)


/opt/app-root/lib64/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/app-root/lib64/python3.9/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue:

Exporting model as ONNX


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

...

Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
/opt/app-root/lib64/python3.9/site-packages/transformers/models/whisper/modeling_whisper.py:1016: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Py

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



/opt/app-root/lib64/python3.9/site-packages/transformers/models/whisper/modeling_whisper.py:1476: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



Weight deduplication check in the ONNX export requires accelerate. Please install accelerate to run it.
		-[x] values not close enough, max diff: 0.01586604118347168 (atol: 0.001)
The ONNX export succeeded with the warning: The maximum absolute difference between the output of the reference model and the ONNX exported model is not within the set tolerance 0.001:
- last_hidden_state: max diff = 0.01586604118347168.
 The exported model was saved at: models
